# Import des bibliothèques

In [37]:
import numpy as np
from scipy import optimize
import pandas as pd
from scipy.optimize import root

# Définition des trois modèles

## 2SFCA :

$$
W_{i j}=\frac{1}{d_{i j}} \hspace{5mm} \text{et} \hspace{5mm} \mathbb{P}_{i j} \propto W_{i j} \hspace{2.5mm} \text{ i.e } \hspace{2.5mm} \mathbb{P}_{i j}=\frac{W_{i j}}{\sum_{k} W_{i k}}
$$

In [38]:
def Prob_eval_SFCA2(d):
    """
    Calcule la probabilité qu'un patient d'une commune i visite un médecin d'une commune j
    en utilisant le modèle SFCA2.

    Args:
        d (numpy.array): Une matrice de distances. L'élément [i, j] représente la distance
            entre la commune i et la commune j.

    Returns:
        Prob (numpy.array): Une matrice de probabilités de même dimension que d. L'élément [i, j] 
            représente la probabilité qu'un patient de la commune i visite un médecin de la commune j.
    """
    W = 1.0 / d
    Prob = W / np.sum(W, axis=1, keepdims=True)
    
    return Prob



## 3SFCA :

$$
W_{i j}=\frac{1}{d_{i j}} \hspace{5mm} \text { et } \hspace{5mm} \mathbb{P}_{i j} \propto W_{i j} S_{j} \hspace{2.5mm} \text { i.e } \hspace{2.5mm} \mathbb{P}_{i j}=\frac{W_{i j} S_{j}}{\sum_{k} W_{i k} S_{k}}
$$


In [39]:
def Prob_eval_SFCA3(d, S):
    """
    Calcule la matrice de probabilité qu'un patient d'une commune i visite un médecin 
    d'une commune j en utilisant le modèle SFCA3.
    La fonction prend comme entrée une matrice de distances, un vecteur de demandes d'offres 
    par commune, et un vecteur d'offres de soins par commune. Elle retourne une matrice 
    de probabilités correspondante.

    Args:
        d (numpy.array): Une matrice de distances. L'élément [i, j] représente 
            la distance entre la commune i et la commune j.
        S (numpy.array): Un vecteur de l'offre de soins pour chaque commune j.

    Returns:
        Prob (numpy.array): Une matrice de probabilités de même dimension que d. L'élément [i, j] 
            représente la probabilité qu'un patient de la commune i visite un médecin de la commune j.
    """
    W = 1.0 / d
    WS = W * S
    Prob = WS / np.sum(WS, axis=1, keepdims=True)
    return Prob

## Point Fixe :
$$
W_{i j}=\frac{1}{d_{i j}} \hspace{5mm} \text { et } \hspace{5mm} \mathbb{P}_{i j} \propto W_{i j} R_{j} \hspace{2.5mm} \text { i.e } \hspace{2.5mm} \mathbb{P}_{i j}=\frac{W_{i j} R_{j}}{\sum_{k} W_{i k} R_{k}}
$$

### Définition des fonctions du point fixe

$$
F: \left\{ 
\begin{array}{cl}
\mathbb{R}^{J} \times \mathbb{R}^{N \times J} & \rightarrow \mathbb{R}^{J} \times \mathbb{R}^{N \times J} \\
\left( R=\left(R_{j}\right)_{j}, \mathbb{P}=\left(\mathbb{P}_{i j}\right)_{i j} \right) & \mapsto \left( F_{1}(\mathbb{P}), F_{2}(R) \right) \\
\end{array} 
\right.
$$

où $ F_{1}(\mathbb{P})=\left(\frac{S_{j}}{\sum_{i} P_{i} \mathbb{P}_{i j}}\right)_{j} $ et  $ F_{2}(R)=\left(\frac{R_{j} W_{i j}}{\sum_{k} R_{k} W_{i k}}\right)_{i j} $


In [12]:
def calc_Rj(P, Prob, S):
    """
    Calcule le vecteur R où chaque élément R[j] est donné par la formule :
    R_j = S_j / (sum_i P_i * Prob_{i,j}).


    Args:
        P (numpy.array): Un vecteur de la demande d'offre pour chaque commune.
        Prob (numpy.array): Une matrice de probabilités. L'élément [i, j] représente 
            la probabilité qu'un patient de la commune i visite un médecin de la commune j.
        S (numpy.array): Un vecteur de l'offre de soins pour chaque commune.

    Returns:
        numpy.array: Un vecteur R. Chaque élément R[j] est donné par la formule : 
            R_j = S_j / (sum_i P_i * Prob_{i,j}).
    """
    R = S / np.sum(P[:, np.newaxis] * Prob, axis=0)
    return R

import numpy as np

def calc_Prob_ij(R, W):
    """
    Calcule la matrice de probabilités Prob où chaque élément Prob[i, j] selon le modele du point fixe donné par la formule :
    Prob[i, j] = R[j] * W[i, j] / (sum_k R[k] * W[i, k]).

    Args:
        R (numpy.array): Un vecteur des offres totales par patients potentiels pour chaque commune.
        W (numpy.array): Une matrice de perméabilité. L'élément [i, j] représente la perméabilité 
            entre la commune i et la commune j.

    Returns:
        numpy.array: Une matrice de probabilités de même dimension que W. L'élément [i, j] 
            représente la probabilité.
    """
    RW = R * W
    Prob = RW / np.sum(RW, axis=1, keepdims=True)
    return Prob

def F(R, Prob, W, P, S):
    """
    Calcule le nouveau R et Prob en utilisant les fonctions calc_Rj et calc_Prob_ij.

    Args:
        R (numpy.array): Un vecteur des offres totales par patients potentiels pour chaque commune.
        Prob (numpy.array): Une matrice de probabilités.
        W (numpy.array): Une matrice des coefficients de perméabilité.
        P (numpy.array): Un vecteur des demandes (nombre de patients dans la région i).
        S (numpy.array): Un vecteur des offres (par exemple, le nombre d'heures de travail des médecins dans la région j).

    Returns:
        R_new (numpy.array): Le nouveau vecteur R calculé par la fonction calc_Rj.
        Prob_new (numpy.array): La nouvelle matrice de probabilités calculée par la fonction calc_Prob_ij.
    """
    Prob_new = calc_Prob_ij(R, W)
    R_new = calc_Rj(P, Prob, S)
    return R_new, Prob_new


### Méthode 1 :
algorithme classique

In [44]:
def Point_fixe_SFCA(W, S, P, maxiter = 10000):
    """
    Implémentation de l'algorithme pour calculer le vecteur R et la matrice Prob avec la méthode du point fixe.

    Args:
        W (numpy.array): Une matrice des coefficients de perméabilité.
        S (numpy.array): Un vecteur des offres (par exemple, le nombre d'heures de travail des médecins dans la région j).
        P (numpy.array): Un vecteur des demandes (nombre de patients dans la région i).
        maxiter (int): Le nombre l'itération maximale.

    Returns:
        R (numpy.array): Un vecteur des offres totales par patients potentiels.
        Prob (numpy.array): Une matrice de probabilité de connexions.
    """

    R = np.ones_like(S)

    for _ in range(maxiter):
        
        P_prime = R * W
        P_prime_row_sum = np.sum(P_prime, axis=1, keepdims=True)
        Prob = P_prime / P_prime_row_sum

        R = S / np.dot(P.T, Prob)

    return R, Prob

In [45]:
def find_fixed_point(F, W, P, S, max_iter=10000):
    """
    Trouve le point fixe de la fonction F.

    Args:
        F (function): La fonction pour laquelle trouver le point fixe.
        W (numpy.array): Une matrice des coefficients de perméabilité.
        P (numpy.array): Un vecteur des demandes (nombre de patients dans la région i).
        S (numpy.array): Un vecteur des offres (par exemple, le nombre d'heures de travail des médecins dans la région j).
        tol (float): La tolérance pour la convergence.
        max_iter (int): Le nombre maximal d'itérations.

    Returns:
        R (numpy.array): Le vecteur R au point fixe.
        Prob (numpy.array): La matrice de probabilités au point fixe.
    """

    R = np.ones_like(S)
    Prob = np.random.rand(len(P), len(S))

    for _ in range(max_iter) :
        R_new, Prob_new = F(R, Prob, W, P, S)
        R = R_new
        Prob = Prob_new


    return R, Prob


### Méthode 2 : 
En utilisant Scipy

In [42]:
def F_diff(X, W, P, S):
    R, Prob = X[:len(S)], X[len(S):].reshape(W.shape)
    R_new, Prob_new = F(R, Prob, W, P, S)
    return np.concatenate([R_new - R, (Prob_new - Prob).ravel()])

def find_fixed_point_2(F, W, P, S, tol=1e-10, max_iter=10000):
    R_init = np.ones_like(S)
    Prob_init = np.random.rand(len(P), len(S))
    X_init = np.concatenate([R_init, Prob_init.ravel()])
    sol = root(F_diff, X_init, args=(W, P, S))
    if sol.success:
        X_fixed = sol.x
        R_fixed, Prob_fixed = X_fixed[:len(S)], X_fixed[len(S):].reshape(W.shape)
        return R_fixed, Prob_fixed
    else:
        print("La méthode n'a pas convergé.")



In [33]:
# Création de données aléatoires  # Pour rendre les résultats reproductibles
W = np.random.rand(5, 5)
S = np.random.rand(5)
P = np.random.rand(5)

In [46]:
R_fixed,Prob_fixed = find_fixed_point_2(F, W, P, S)
print(np.linalg.norm(F(R_fixed, Prob_fixed, W, P, S)[0] - R_fixed))
print(np.linalg.norm(F(R_fixed, Prob_fixed, W, P, S)[1] - Prob_fixed))

7.208596235890075e-10
4.6685244435798397e-11


In [47]:
R_fixed,Prob_fixed = Point_fixe_SFCA(W, S, P)
print(np.linalg.norm(F(R_fixed, Prob_fixed, W, P, S)[0] - R_fixed))
print(np.linalg.norm(F(R_fixed, Prob_fixed, W, P, S)[1] - Prob_fixed))

3.1401849173675503e-16
2.0301975233231525e-07


In [48]:
R_fixed,Prob_fixed = find_fixed_point(F, W, P, S)
print(np.linalg.norm(F(R_fixed, Prob_fixed, W, P, S)[0] - R_fixed))
print(np.linalg.norm(F(R_fixed, Prob_fixed, W, P, S)[1] - Prob_fixed))

0.00026858115593428783
1.6637534428730296e-06


In [34]:
print("Méthode 1: Itération jusqu'à la convergence")
R_fixed, Prob_fixed = find_fixed_point(F, R_init, Prob_init, W, P, S)
print(F(R_fixed, Prob_fixed, W, P, S)[0] - R_fixed)
print(F(R_fixed, Prob_fixed, W, P, S)[1] - Prob_fixed)

Méthode 1: Itération jusqu'à la convergence
[-2.42910211e-05 -1.97353350e-04  2.20411863e-04  9.05396329e-06
 -1.57235549e-06]
[[-2.48842571e-06 -7.59249851e-05  7.83374489e-05  1.26092762e-06
  -1.18496575e-06]
 [-1.03770561e-05 -6.91901318e-05  7.97956746e-05  1.33297088e-06
  -1.56145766e-06]
 [-6.53914752e-06 -2.83536439e-05  2.84272234e-05  6.65406298e-06
  -1.88495012e-07]
 [-1.52595764e-05 -5.79572988e-05  7.29294478e-05  7.37913948e-07
  -4.50486530e-07]
 [-2.70796802e-06 -1.09381751e-04  1.01982056e-04  9.09861534e-06
   1.00904751e-06]]


In [35]:
R_fixed, Prob_fixed = Point_fixe_SFCA(W, S, P)
print(F(R_fixed, Prob_fixed, W, P, S)[0] - R_fixed)
print(F(R_fixed, Prob_fixed, W, P, S)[1] - Prob_fixed)

[ 0.00000000e+00 -2.22044605e-16  0.00000000e+00  0.00000000e+00
 -2.22044605e-16]
[[-2.13040272e-09 -6.49924863e-08  6.70630986e-08  1.07827909e-09
  -1.01848874e-09]
 [-8.88389450e-09 -5.92270730e-08  6.83113165e-08  1.13766352e-09
  -1.33801267e-09]
 [-5.59784628e-09 -2.42702320e-08  2.43365657e-08  5.69411845e-09
  -1.62606040e-10]
 [-1.30639398e-08 -4.96117476e-08  6.24337188e-08  6.31166411e-10
  -3.89197952e-10]
 [-2.31933847e-09 -9.36368280e-08  8.73056834e-08  7.78694588e-09
   8.63537283e-10]]


In [6]:
point_fixe = optimize.fixed_point(f, x0 = -10, maxiter = 10000000)
print(f"Le point fixe est: {point_fixe}")

NameError: name 'f' is not defined

In [47]:
S = np.array([1,2])
P = np.array([2,1])
Prob = Prob_eval_SFCA3(d, S)

In [48]:
P

array([2, 1])

In [49]:
S

array([1, 2])

In [50]:
Prob

array([[0.5, 0.5],
       [0.2, 0.8]])

In [56]:
R = calc_Rj(P, Prob, S)
R

array([0.83333333, 1.11111111])

In [57]:
R * Prob

array([[0.41666667, 0.55555556],
       [0.16666667, 0.88888889]])

In [54]:
2/1.8

1.1111111111111112